In [1]:
!nvidia-smi

Thu Apr 10 16:31:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:89:00.0 Off |                    0 |
| N/A   26C    P0             39W /  300W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from bert_score import score as bert_score

In [3]:
# %%capture
# # !pip install datasets==1.2.1
# # !pip install evaluate
# !pip install evaluate[rouge]
# !pip install rouge_score absl-py



# # !pip install transformers==4.2.0
# # !pip install rouge_score

In [4]:
# !pip install ipywidgets notebook tqdm


In [5]:
# !pip install rouge_score

## Evaluation

In [1]:
import pandas as pd
import json

df1 = pd.read_csv('data/all_articles.csv')
df1

,article_id,title,abstract,full_text
0,PMC310413,Copy Number Variation Analysis in Single-Sutur...,"<S> in this study, we hypothesize that rare co...",craniosynostosis is defined as the premature f...
1,PMC829490,A retrospective cohort study of disease-relate...,<S> abstract\ncentral venous catheters (cvc) a...,hospital-acquired venous thromboembolism (vte)...
2,PMC278216,Hypereosinophilia Presenting as Eosinophilic V...,<S> a 32-yr-old korean woman presented to us w...,hypereosinophilia is an abnormal condition def...
3,PMC954564,Childhood appendicitis and future risk of infl...,<S> abstract\n\naim\nthe aim of this study was...,the aim of this study was to investigate the a...
4,PMC489257,Modeling the Dynamics of High-Grade Serous Ova...,<S> high-grade serous ovarian cancer (hgsoc) r...,"ovarian cancer is a relatively rare disease, r..."
...,...,...,...,...
74768,PMC496084,Music therapy in Huntington’s disease: a proto...,<S> background\nhuntington’s disease is a prog...,"huntington’s disease is a progressive, neurode..."
74769,PMC611943,Health Canada’s use of expedited review pathwa...,<S> objectives\nthis study examines the use of...,this study examines the use of expedited appro...
74770,PMC538305,Amplitude of low-frequency fluctuations in fir...,<S> despite different treatments and courses o...,differences in clinical features between major...
74771,PMC360398,Fine Mapping for Weaver Syndrome in Brown Swis...,<S> bovine progressive degenerative myeloencep...,bovine progressive degenerative myeloencephalo...


In [ ]:
# import ast
# df1['article_text'] = df1['full_text'].apply(lambda x: ast.literal_eval(x))
# df1['abstract_text'] = df1['abstract'].apply(lambda x: ast.literal_eval(x))
# df1['section_names'] = df1['section_names'].apply(lambda x: ast.literal_eval(x))
# df1['sections'] = df1['sections'].apply(lambda x: ast.literal_eval(x))
# df1

SyntaxError: invalid syntax (<unknown>, line 1)

In [4]:
df1['article'] = df1['full_text'] #.apply(lambda x: ' '.join(x))
df1['abstract'] = df1['abstract'] #.apply(lambda x: ' '.join(x))
df1

,article_id,title,abstract,full_text,article
0,PMC310413,Copy Number Variation Analysis in Single-Sutur...,"<S> in this study, we hypothesize that rare co...",craniosynostosis is defined as the premature f...,craniosynostosis is defined as the premature f...
1,PMC829490,A retrospective cohort study of disease-relate...,<S> abstract\ncentral venous catheters (cvc) a...,hospital-acquired venous thromboembolism (vte)...,hospital-acquired venous thromboembolism (vte)...
2,PMC278216,Hypereosinophilia Presenting as Eosinophilic V...,<S> a 32-yr-old korean woman presented to us w...,hypereosinophilia is an abnormal condition def...,hypereosinophilia is an abnormal condition def...
3,PMC954564,Childhood appendicitis and future risk of infl...,<S> abstract\n\naim\nthe aim of this study was...,the aim of this study was to investigate the a...,the aim of this study was to investigate the a...
4,PMC489257,Modeling the Dynamics of High-Grade Serous Ova...,<S> high-grade serous ovarian cancer (hgsoc) r...,"ovarian cancer is a relatively rare disease, r...","ovarian cancer is a relatively rare disease, r..."
...,...,...,...,...,...
74768,PMC496084,Music therapy in Huntington’s disease: a proto...,<S> background\nhuntington’s disease is a prog...,"huntington’s disease is a progressive, neurode...","huntington’s disease is a progressive, neurode..."
74769,PMC611943,Health Canada’s use of expedited review pathwa...,<S> objectives\nthis study examines the use of...,this study examines the use of expedited appro...,this study examines the use of expedited appro...
74770,PMC538305,Amplitude of low-frequency fluctuations in fir...,<S> despite different treatments and courses o...,differences in clinical features between major...,differences in clinical features between major...
74771,PMC360398,Fine Mapping for Weaver Syndrome in Brown Swis...,<S> bovine progressive degenerative myeloencep...,bovine progressive degenerative myeloencephalo...,bovine progressive degenerative myeloencephalo...


In [5]:
import re

def clean_special_tokens(text):
    """
    把 <S> </S> <pad> <unk> <mask> 等各種模型用 tag 全部清掉
    如果不是 str，直接回空字串（防爆）
    """
    if not isinstance(text, str):
        return ''
    # 清掉所有 <...> 標籤
    cleaned = re.sub(r"<[^>]+>", "", text)
    # 清多餘空白
    cleaned = cleaned.strip()
    return cleaned


In [6]:
df1['abstract'] = df1['abstract'].apply(clean_special_tokens)
df1['abstract']

0        in this study, we hypothesize that rare copy n...
1        abstract\ncentral venous catheters (cvc) are w...
2        a 32-yr-old korean woman presented to us with ...
3        abstract\n\naim\nthe aim of this study was to ...
4        high-grade serous ovarian cancer (hgsoc) repre...
                               ...                        
74768    background\nhuntington’s disease is a progress...
74769    objectives\nthis study examines the use of exp...
74770    despite different treatments and courses of il...
74771    bovine progressive degenerative myeloencephalo...
74772    studies evaluating the association between age...
Name: abstract, Length: 74773, dtype: object

In [11]:
label_columns = [
    "letter",
    "editorial",
    "comment",
    "case_reports",
    "published_erratum",
    "review",
    "news",
    "historical_article",
    "biography",
    "clinical_studies_as_topic",
    "clinical_trials_as_topic",
]

label_json_list = [] 

for label in label_columns:
    filtered_df = df1[df1[label] == 1].copy()
    filtered_df = filtered_df[['article', 'abstract']]  
    filtered_df['label_name'] = label  

    record_list = filtered_df.to_dict(orient='records')
    label_json_list.append(record_list)


In [12]:
# 排序 + 移除空 list 的版本
label_json_list = sorted(
    [record_list for record_list in label_json_list if len(record_list) > 0],
    key=len,
    reverse=True
)


In [13]:
import random

random.seed(42)  

max_per_label = 250

label_json_list_trimmed = [
    random.sample(records, min(len(records), max_per_label))
    for records in label_json_list
]



In [14]:
label_json_list_trimmed[0]

[{'article': 'a paranasal mucocele is usually defined as an accumulation of mucosal secretions into a paranasal sinus with obstruction caused by inflammation , fibrosis , trauma , previous surgery , anatomical abnormality , or a mass lesion such as an osteoma . when the mucocele becomes infected it is called a pyomucocele or pyocele , which contains pus partially or fully in cysts . the clinical manifestation of mucoceles is benign in most cases [ 5 , 6 ] . most mucoceles arise in the frontal and ethmoid sinuses , but intracranial extension is uncommon [ 3 , 5 ] . in this paper , we describe a case of a giant pyomucocele with extension into the anterior cranial fossa 32 years after a primary frontal sinus fracture due to a traffic accident . a 51-year - old woman presenting with left exophthalmos and ophthalmoplegia associated with large swelling over the left orbit and frontal region during the last three months was referred to the department of ophthalmology of hokkaido university ho

In [15]:
from transformers import LEDTokenizer, LEDForConditionalGeneration
# load tokenizer
import torch
from evaluate import load as load_metric
tokenizer = LEDTokenizer.from_pretrained("patrickvonplaten/led-large-16384-pubmed")
model = LEDForConditionalGeneration.from_pretrained("patrickvonplaten/led-large-16384-pubmed").to("cuda").half()


def generate_answer(batch):
    inputs_dict = tokenizer(
        batch["article"],
        padding="max_length",
        max_length=8192,
        return_tensors="pt",
        truncation=True
    )
    
    input_ids = inputs_dict.input_ids.to("cuda")
    attention_mask = inputs_dict.attention_mask.to("cuda")
    
    # 全域注意力放在 <s> token 上
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    with torch.no_grad():  # 預測不需要梯度
        predicted_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
            max_length=512,  # 你可以調整這個摘要長度
            num_beams=4,     # beam search 的數量，也可調
        )
    
    predicted_abstracts = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
    
    return {"predicted_abstract": predicted_abstracts}



tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

In [16]:
from datasets import Dataset
from tqdm import tqdm
import numpy as np

def avg_metric(metric_list, key):
    return np.mean([m[key] for m in metric_list])
    
rouge = load_metric("rouge")

In [17]:

from bert_score import score as bert_score
rows = []

for record_list in tqdm(label_json_list_trimmed):
    if record_list == []:
        continue
    dataset = Dataset.from_list(record_list)  # 轉成 HuggingFace Dataset
    dataset = dataset.map(lambda x: {
        "article": x["article"],
        "abstract": x["abstract"]
    })
    # print(1, dataset.column_names)

    # 跑模型產生摘要
    dataset = dataset.map(generate_answer, batched=True, batch_size=4, remove_columns=[], load_from_cache_file=False)
    # print(2, dataset.column_names)
    # 清預測結果
    dataset = dataset.map(lambda x: {
        "predicted_abstract": clean_special_tokens(x["predicted_abstract"])
    }, remove_columns=[], load_from_cache_file=False)
    # print(3, dataset.column_names)
    
    # 計算 ROUGE
    scores = rouge.compute(
        predictions=dataset["predicted_abstract"],
        references=dataset["abstract"],
        rouge_types=["rouge1", "rouge2", "rougeL"],
        # use_aggregator=False,
        # use_stemmer=True
    )
    P, R, F1 = bert_score(
        cands=dataset["predicted_abstract"],
        refs=dataset["abstract"],
        lang="en",  # 根據你資料語言調整，支援 'zh', 'en', 'fr', 等
        model_type="distilbert-base-uncased", 
        rescale_with_baseline=True  # 建議開啟以利跨模型比較
    )
    
    row = {
        "labels": record_list[0]['label_name'],
        "rouge-1": np.mean(scores["rouge1"]),
        "rouge-2": np.mean(scores["rouge2"]),
        "rouge-L": np.mean(scores["rougeL"]),
        "bert-P": P.mean().item(),
        "bert-R": R.mean().item(),
        "bert-F1": F1.mean().item()
    }

    rows.append(row)

rouge_df = pd.DataFrame(rows)


  0%|          | 0/11 [00:00<?, ?it/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...


 91%|█████████ | 10/11 [1:19:20<05:08, 308.07s/it]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]


100%|██████████| 11/11 [1:19:25<00:00, 433.25s/it]


In [18]:
rouge_df = pd.DataFrame(rows)
rouge_df

,labels,rouge-1,rouge-2,rouge-L,bert-P,bert-R,bert-F1
0,case_reports,0.225812,0.053250,0.146505,0.123670,0.163165,0.139183
1,review,0.289952,0.069907,0.173700,0.253205,0.244593,0.247277
2,comment,0.165208,0.019855,0.112233,-0.056781,0.073388,0.003784
3,clinical_studies_as_topic,0.291569,0.074345,0.169333,0.227125,0.259275,0.240994
4,clinical_trials_as_topic,0.288622,0.072377,0.167927,0.227216,0.248697,0.236189
5,historical_article,0.247405,0.060384,0.153379,0.126590,0.227108,0.172807
6,editorial,0.201997,0.031694,0.134062,0.034825,0.130440,0.078688
7,letter,0.205371,0.032078,0.127497,0.054157,0.113934,0.081044
8,biography,0.233750,0.050527,0.145757,0.063837,0.156918,0.106627
9,news,0.156303,0.025412,0.116096,-0.165166,0.011866,-0.087235


In [19]:
rouge_df.to_csv("data/rouge_scores2.csv", index=False)


## all

In [20]:
rouge_df = pd.read_csv("data/rouge_scores2.csv")

In [21]:
import random

random.seed(42)  

max_per_label = 250

filtered_df = df1[['article', 'abstract']]  
filtered_df

,article,abstract
0,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...
1,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...
2,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...
3,agenesis of the inferior vena cava ( ivc ) as ...,background : agenesis of the inferior vena cav...
4,a review of the literature and an extensive me...,backgrounda review of the literature and an ex...
...,...,...
41849,mechanical valve thrombosis ( mvt ) may cause ...,"in the present study , the authors investigate..."
41850,squamous cell carcinoma ( scc ) of the thyroid...,primary squamous cell carcinoma of the thyroid...
41851,although levodopa ( ld ) still remains the gol...,the aim of this article is to provide a short ...
41852,fibromyalgia is a chronic pain syndrome of unk...,fibromyalgia is a chronic pain syndrome charac...


In [22]:


all_record = random.sample(filtered_df.to_dict(orient='records'), min(len(filtered_df), max_per_label))


dataset = Dataset.from_list(all_record)  # 轉成 HuggingFace Dataset
dataset = dataset.map(lambda x: {
    "article": x["article"],
    "abstract": x["abstract"]
})
# print(1, dataset.column_names)

# 跑模型產生摘要
dataset = dataset.map(generate_answer, batched=True, batch_size=4, remove_columns=[], load_from_cache_file=False)
# print(2, dataset.column_names)
# 清預測結果
dataset = dataset.map(lambda x: {
    "predicted_abstract": clean_special_tokens(x["predicted_abstract"])
}, remove_columns=[], load_from_cache_file=False)
# print(3, dataset.column_names)

# 計算 ROUGE


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [23]:
scores = rouge.compute(
    predictions=dataset["predicted_abstract"],
    references=dataset["abstract"],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    # use_aggregator=False,
    # use_stemmer=True
)


P, R, F1 = bert_score(
    cands=dataset["predicted_abstract"],
    refs=dataset["abstract"],
    lang="en",  # 根據你資料語言調整，支援 'zh', 'en', 'fr', 等
    rescale_with_baseline=True,  # 建議開啟以利跨模型比較
    model_type="distilbert-base-uncased", 
)

row = {
    "labels": 'all',
    "rouge-1": np.mean(scores["rouge1"]),
    "rouge-2": np.mean(scores["rouge2"]),
    "rouge-L": np.mean(scores["rougeL"]),
    "bert-P": P.mean().item(),
    "bert-R": R.mean().item(),
    "bert-F1": F1.mean().item()
}

rouge_df.loc[len(rouge_df)] = row
rouge_df.to_csv("data/rouge_scores2.csv", index=False)